<h1 align='center'>Data workflow</h1>

<h3 align='center'>Laura G. Funderburk</h3>

<h3 align='center'>Data Scientist, Cybera</h3>

<h2 align='center'>3 Data Workflow Practices I follow</h2>


1. Leverage both Bash and Python (or R) to access and process data

2. Don't be afraid of messy exploratory work

3. ...but be sure to tidy up as you go!

<h2 align='center'>Leverage both Bash and Python (or R) to access and process data</h2>


To motivate this exercise, I will showcase how we can leverage these tools to write a set of scripts that:

1. Download data from two different API containing data on CO2 emissions. 

2. Time downloads. 

3. Generate CSV with query output and time to download.

4. Generate visualizations of the time it took two different APIs to download the data.

5. Push results to a GitHub repository.

<h2 align='center'>Don't be afraid of messy exploratory work</h2>

The scripts with final work look nice...so to motivate how I got there, here is some of what my initial exploration looked like. 

<h2 align='center'>Example: pulling currency data from a REST API</h2>


https://api.carbonintensity.org.uk/

Sample query I: getting factors contributing to CO2

`curl https://api.carbonintensity.org.uk/intensity/factors`

Sample query II: getting actual and forecasted CO2 intensity in January 1 2021

`curl https://api.carbonintensity.org.uk/intensity/date/2021-01-01`

Sample query III: getting

`curl https://api.carbonintensity.org.uk/regional/intensity/2021-01-01/2021-06-01/regionid/1`

1. North Scotland
2. South Scotland
3. North West England
4. North East England
5. South Yorkshire
6. North Wales, Merseyside and Cheshire
7. South Wales
8. West Midlands
9. East Midlands
10. East England
11. South West England
12. South England
13. London
14. South East England
15. England
16. Scotland
17. Wales

In [ ]:
# Code to curl data using Python
import requests
import pandas as pd
import time
import plotly.express as px

# Get codes
factors_url = "https://api.carbonintensity.org.uk/intensity/factors"
factors_response = requests.get(factors_url)
factors_response.raise_for_status()
factors_jsonResponse = factors_response.json()

In [ ]:
print(factors_jsonResponse)

display(pd.json_normalize(factors_jsonResponse, record_path='data'))



Suppose we want to download all daily data between November 1 2020 and July 1 2021? We'd have to repeat code over, and over and over again. 

We will create a function instead!


In [ ]:
def perform_query(url):
    start_time = time.time()
    # Using GET command 
    response = requests.get(url)
    total_time = time.time() - start_time
    
    # Raise issues if response is different from 200
    response.raise_for_status()
    
    # access JSOn content
    jsonResponse = response.json()
    
    return jsonResponse

We then document our function, and make it more robust. 

In [ ]:
def perform_query(url):
    """
    This function performs a query on a REST API and returns a dataframe with the content 
    of the query
    
    Parameters
    ----------
        url (str) full URL containing the query
        record_flag (bool)
        
    Returns
    -------
        flat_df (dataframe object) containing JSON response in dataframe format
    """
    
    try:
        # Time query
        start_time = time.time()
        # Using GET command 
        response = requests.get(url)
        total_time = time.time() - start_time
        print(total_time)
        # Raise issues if response is different from 200
        response.raise_for_status()

        # access JSOn content
        jsonResponse = response.json()

        return jsonResponse
    
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

We can then use function and iterate.

Rembember:

1. Get your function to do one thing at a time

2. Document your functions (tell us what your function does, add variable and function names that are meaningful)

3. Ensure your functions are robust - test what happens on edge cases and handle those cases.

In [ ]:
# Create data range 
date_range = pd.date_range(start="2019-01-01", end="2019-01-01", freq='D')
dates = [date.date() for date in date_range]
# Iterate over all dates 
# Perform query
# Store content into master dataframe
all_df = []
for item in dates:
    new_url = f'https://api.carbonintensity.org.uk/intensity/date/{item}'
    query = perform_query(new_url)
    df = pd.json_normalize(query, record_path="data")
    all_df.append(df)


In [ ]:
def download_intensity_by_date(start_d, end_d):
    
    """
    This function downloads actual and forecasted intensity during a given time range
    
    Parameters
    ----------
    start_d (str) date in format YYYY-MM-DD indicating start of observations
    end_d (str) date in format YYYY-MM-DD indicating end of observations
    
    Returns
    -------
    
    """
    try:
        # Create data range 
        date_range = pd.date_range(start=start_d, end=end_d, freq='D')
        dates = [date.date() for date in date_range]
        # Iterate over all dates 
        # Perform query
        # Store content into master dataframe
        all_df = []
        for item in dates:
            new_url = f'https://api.carbonintensity.org.uk/intensity/date/{item}'
            query = perform_query(new_url)
            df = pd.json_normalize(query, record_path="data")
            all_df.append(df)


        return all_df
    except:
        print("Error")

Try it out!!!

In [ ]:
start="2019-12-01"
end="2019-12-10"
all_df = download_intensity_by_date(start, end)

master_sheet = pd.concat(all_df)
display(master_sheet.head())
print(master_sheet.info())

Once your function is ready and tested, bring it to your script. 



In [ ]:
# Time formatting
master_sheet['from.date'] = pd.to_datetime(master_sheet['from'], format='%Y-%m-%d')
master_sheet['to.date'] = pd.to_datetime(master_sheet['to'], format='%Y-%m-%d')

# daily average
davg_df2 = master_sheet[["from.date",\
                         'intensity.actual',\
                         'intensity.forecast']].groupby([pd.Grouper(freq='D', key='from.date')]).mean()


In [ ]:
davg_df2

In [ ]:
px.scatter(data_frame=davg_df2, 
           x=davg_df2.index, 
           y='intensity.actual',
          title="Recorded intensity (2019 - 2021)",
          labels = {'x':"Date", "intensity.actual": "Recorded intensity"})

In [ ]:
px.scatter(data_frame=davg_df2, 
           x=davg_df2.index, 
           y='intensity.forecast',
          title="Recorded intensity (2019 - 2021)",
          labels = {'x':"Date", "intensity.forecast": "Forecasted intensity"})


In [ ]:
px.scatter(data_frame=davg_df2, 
           x='intensity.forecast', 
           y='intensity.actual',
          title="Intensity: actual vs forecasted",
          labels = {'intensity.recorded':"Forecasted intensity", "intensity.actual": "Recorded intensity"},
          hover_name=davg_df2.index)

In [ ]:
# Demonstrate bringing our Python script into Bash and automate
# Let's explore building a script together

<h2 align='center'>...but be sure to tidy up as you go!</h2>

1. Ensure to give functions and variables descriptive names

2. Document what your code does, add notes on specific data types any functions you write take and return, along with meaningful names

3. Test that your code does what you think it does

4. **R**efactor code, **R**educe repetition, **R**emove unused code (no need to keep a large piece of commented code "in case I need it some day"). 

<h2 align='center'>Data Collaboration Practices I follow</h2>


1. When you write code, assume another person at some point in the future will review it. Write your code as if you are writing an article for someone else to read. 

2. GitHub etiquette to contribute code: fork a repository, create a new branch, make changes on that branch, create a pull request. Create clear notes on what the contribution does. 

    **Pro-tip: read a repository's issues and comment on them proposing your changes before you invest time creating something that might be a duplicate effort, or which is not compatible.**
    
3. GitHub etiquette to request code changes: create **clear**, **concise**, **well documented**, and **specific** issues and documentation on how members can interact with and add content to the repository. 

4. Do your best to provide construtive feedback, and assume that when someone provides feedback to you, they want to help you improve the quality of code/documentation/feature.

<h2 align='center'>Hands on exercise contributing to a repository</h2>

1. Visit https://github.com/cybera/DS-industry-fellowship-2021

2. View issues https://github.com/cybera/DS-industry-fellowship-2021/issues

3. Hands on time: breaking into two teams -> each team works together on one of the two issues


<h2 align='center'>Hands on exercise contributing to a repository</h2>


In these issues there are tasks for improving code quality in a python script called `dummy.py`. 

Let's first get familiar with what the code does.

In [ ]:
%run -i ./scripts/dummy.py

The function sample_function will take two numbers (a,b) and perform the equivalent of taking $b^2$. 

For example, if $a = 2$ and $b = 3$, the function will compute $3^2 = 9$

|`i` |`j` | `iter_sum`|
|-|-|-|
|0 |0 |0|
|0 |1 |1|
|0 |2 |3|
|1 |0 |4|
|1 |1 |6|
|1 |2 |9|

Our scripts time both approaches.

At the end of the loop, `iter_summ` returns 9.

<h2 align='center'>Summary of what we learned</h2>
